In [1]:
import pandas as pd
import numpy as np
import requests
import base64
import json
from bs4 import BeautifulSoup

For couple of IMDB movie ids, lets us scrap related data

In [2]:
movie_ids = ['tt0111161','tt0088763']

In [3]:
movie_data_list =[]
iteration=0
for i, val in enumerate(movie_ids):
    data ={}
    URL ='https://www.imdb.com/title/'+val+'/'
    resp = requests.get(URL)
    data['IMDB_URL'] = URL

    soup = BeautifulSoup(resp.text, "html.parser")

    title_wrapper = soup.select_one('div.title_wrapper')
    #print(title_wrapper.h1.text)
    if title_wrapper is None:
        continue
    data['IMDB_MOVIENAME'] = title_wrapper.h1.text

    originalTitle = title_wrapper.select_one('div.originalTitle')
    if originalTitle:
        data['OriginalTitle'] = originalTitle.text
    else:
        data['OriginalTitle'] = None

    subText = title_wrapper.select_one('div.subtext')
    if subText:
        data['SubText'] = subText.text.replace('\r','').replace('\n','').replace(' ','')
    else:
        data['SubText'] = None


    rating_div = soup.select('div.ratingValue')
    if rating_div:
        rating_spans = rating_div[0].findAll('span')
        if rating_spans:
            #print(rating_spans[0].text)
            #print(rating_div[0].strong['title'])
            data["IMDBRating"] = rating_spans[0].text
            data["RatingStats"] = rating_div[0].strong['title']



    plot_summary_div = soup.select('div.plot_summary ')
    if plot_summary_div:
        credit_summary_items = plot_summary_div[0].findAll('div',{'class':'credit_summary_item'})
        if credit_summary_items:
            dir_names = [dir_name.text for dir_name in credit_summary_items[0].findAll('a')]
            data[credit_summary_items[0].select_one('h4').text.strip(':').replace('s','')] = dir_names
            if len(credit_summary_items)>1:
                writers = [writer_name.text for writer_name in credit_summary_items[1].findAll('a')]
                data[credit_summary_items[1].select_one('h4').text.strip(':').replace('s','')] = writers


                if len(credit_summary_items)>2:
                    stars = [name.text for name in credit_summary_items[2].findAll('a')[:-1]]
                    data[credit_summary_items[2].select_one('h4').text.strip(':').replace('s','')] = stars


    story_line_div = soup.select('div#titleStoryLine')
    if story_line_div:
        headings = story_line_div[0].findAll("h4")
        #print(headings)
        genres =list()
        for index, val in enumerate(headings):
            if val.text.strip()=='Genres:':

                genre_div = story_line_div[0].select("div")[index+1]
                genres = [val.text.strip() for val in genre_div.select('a')]
                data['GENRES'] = genres
                break
            else:
                data['GENRES'] = None

    divDetail = soup.select('div#titleDetails')

    info ='Country,Language,Also Known As,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Production Co,Runtime,GENRES,Director,Writer,Star,RatingStats,IMDBRating,SubText,OriginalTitle,IMDB_MOVIENAME,IMDB_URL,MOVIENAME'
    info = info.split(',')
    if divDetail:
        for item in divDetail[0].findAll('div'):
            heading_name = item.select('h4')
            if heading_name:
                if heading_name[0].text.strip(':') in info:
                    v = item.text.replace('\r','').replace('\n','').split(':')

                    data[v[0]] = v[1]
    s1 = set(data.keys())
    s2 = set(info)
    not_found = list(s2-s1)
    
    # for consistency place None on for keys where value is not present in the IMSB page
    for key in not_found:
        data[key] = None

    movie_data_list.append(data)
    iteration+=1
    print('\r', iteration, end='')



 2

In [4]:
scraped_df = pd.DataFrame(movie_data_list)

In [5]:
scraped_df.head()

,IMDB_URL,IMDB_MOVIENAME,OriginalTitle,SubText,IMDBRating,RatingStats,Director,Writer,Star,GENRES,Country,Language,Also Known As,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Production Co,Runtime,MOVIENAME
0,https://www.imdb.com/title/tt0111161/,The Shawshank Redemption (1994),None,R|2h22min|Drama|14October1994(USA),9.3,"9.3 based on 2,266,979 user ratings",[Frank Darabont],"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",[Drama],USA,English,Die Verurteilten See more »,"$25,000,000 (estimated)","$727,327,25 September 1994","$28,699,976","$28,815,245",Castle Rock Entertainment See more »,142 min,None
1,https://www.imdb.com/title/tt0088763/,Back to the Future (1985),None,"1h56min|Adventure,Comedy,Sci-Fi|3July1985(USA)",8.5,"8.5 based on 1,020,651 user ratings",[Robert Zemeckis],"[Robert Zemeckis, Bob Gale]","[Michael J. Fox, Christopher Lloyd, Lea Thompson]","[Adventure, Comedy, Sci-Fi]",USA,English,Volver al futuro See more »,"$19,000,000 (estimated)","$11,152,500,7 July 1985","$211,159,762","$391,352,459","Universal Pictures, Amblin Entertainment, U-D...",116 min,None


In [6]:
scraped_df.to_csv('Data/Output/scraped_sample_imdb.csv')